# RUL Prediction with LSTM
b. Xs' and W by flight class
c. Xs' and W and $\theta$

1. Load flight effects model and validate performance
2. set up hyperparameter tuning for RUL prediction network (LSTM)
3. Evaluate different lookback windows or implement adaptive window

In [114]:
import os
import sys
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

base_dir = os.path.dirname(os.getcwd())
print(base_dir)
sys.path.insert(1, base_dir)
from package.api import DB as api
import package.utils as utils
import package.tuning as tuning
utils.check_gpu()

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers, metrics
#import tensorflow_addons as tfa

import keras_tuner as kt

from kerastuner_tensorboard_logger import (
    TensorBoardLogger,
    setup_tb  # Optional
)

paths_df = pd.read_csv(base_dir + '/paths.csv')
paths_df['path'] = base_dir + '/' + paths_df['path']

Fc = 3
dataset = 'DS08'
log_location = base_dir + '/logs'
model_location = base_dir + '/models'
data_location = base_dir + '/data'
data_header = f'Fc-{Fc}_dataset-{dataset}'


%matplotlib inline
%load_ext autoreload
%autoreload 2

/home/macslab/phm2021_data_challenge
2.8.0
[INFO] GPU?: <True> [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
expanding memory growth
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0. Connect to db and load the data

In [2]:
# THESE ARE YOUR CREDENTIALS IN PLAIN TEXT!
params = {'datasource.username': 'macslab', # the username of the logged in user
            'datasource.password': 'Ch0colate!', 
            'datasource.database': 'ncmapss_db', # <- NO CHANGE 
            'datasource.url': '10.2.219.98', # <- or your database installation location
            'datasource.port': '5432'} # <- most likely don't change
#print(params)
db, cur =  api.connect(params)
db.set_session(autocommit=True)
del(params)

[INFO] connecting to db.
[INFO] connected.


In [4]:
units_df = api._get_units(db=db)
units_df.head()

,id,serial_number,age,eol,rul,group_id,Fc,unit,dataset
0,1,xhJzs6e3,0.0,100.0,100.0,1,1,1,DS01-005
1,2,a2wH8lJP,0.0,75.0,75.0,1,3,2,DS01-005
2,3,zwaWwxPl,0.0,100.0,100.0,1,2,3,DS01-005
3,4,ByMDLFeB,0.0,95.0,95.0,1,1,4,DS01-005
4,5,rcedq8Jy,0.0,89.0,89.0,1,3,5,DS01-005


In [12]:
pd.unique(units_df['dataset'])

array(['DS01-005', 'DS03-012', 'DS04', 'DS05', 'DS06', 'DS07',
       'DS08a-009', 'DS08c-008'], dtype=object)

In [92]:
units = units_df[(units_df['Fc'] == Fc) & (units_df['dataset'].str.contains(dataset))]
units

,id,serial_number,age,eol,rul,group_id,Fc,unit,dataset
66,67,DH4gw7GB,0.0,51.0,51.0,1,3,2,DS08a-009
70,71,XTNVUYvx,0.0,58.0,58.0,1,3,6,DS08a-009
76,77,cHjtOlTA,0.0,58.0,58.0,2,3,12,DS08a-009
77,78,K32vLF2c,0.0,56.0,56.0,2,3,13,DS08a-009
78,79,mvqpgmYg,0.0,59.0,59.0,2,3,14,DS08a-009
80,81,VDsiPMEd,0.0,53.0,53.0,1,3,1,DS08c-008
81,82,07Zcf56x,0.0,52.0,52.0,1,3,2,DS08c-008
82,83,p820WHoV,0.0,53.0,53.0,1,3,3,DS08c-008
83,84,36CHLW1W,0.0,48.0,48.0,1,3,4,DS08c-008
84,85,21ljtsPT,0.0,50.0,50.0,1,3,5,DS08c-008


In [19]:
tables = ['summary_tb', 'telemetry_tb']
downsample=20
df = api._get_data(db=db,
                   units=pd.unique(units.id),
                   tables=tables,
                   downsample=downsample).astype(np.float32)
utils.add_time_column(units=pd.unique(units.id), df=df)
utils.add_rul_column(units=pd.unique(units.id), df=df)

## 1. Make Xs_prime and y 

In [46]:
W_cols = ['Mach', 'alt', 'TRA', 'T2', 'time']
Xs_cols = ['Wf', 'Nf', 'Nc', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50']
aux_cols = ['cycle', 'hs', 'Fc', 'asset_id']

model = keras.models.load_model(paths_df[paths_df['name']=='flight_effects'].path.values[0])
yscaler = joblib.load(paths_df[paths_df['name']=='flight_effects_yscaler'].path.values[0])
xscaler = joblib.load(paths_df[paths_df['name']=='flight_effects_xscaler'].path.values[0])

In [47]:
trace = yscaler.transform(df[Xs_cols])
pred = model.predict(xscaler.transform(df[W_cols]))
res = trace - pred
dfx = pd.DataFrame(data=res, columns=Xs_cols)
df_x = pd.DataFrame(data=xscaler.transform(df[W_cols]), columns=W_cols)
dfx = pd.concat([dfx, df_x, df[aux_cols]], axis=1)
dfx['rul'] = df['rul'].values
dfx.time = dfx.time + (dfx.cycle -1)
dfx0 = dfx[dfx.hs == 0]

2022-02-18 08:00:49.484710: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


## 2. split sets and view feature distributions

In [100]:
train_df, train_y, val_df, val_y, test_df, test_y = utils.train_test_split(df=dfx0, units=list(pd.unique(units.id)), y_labels=['rul'], t_labels=aux_cols + Xs_cols + W_cols, train_pct = 1.0, val_pct=0.0, test_pct=0.0, verbose=True)

10
10
0
0
train, val, test set counts: 10, 0, 0
10 10
train units: [77, 85, 81, 71, 83, 84, 79, 67, 82, 78]
val units: []
test units: []


In [101]:
train_y = pd.DataFrame(data=train_y, columns=['rul'])
val_y = pd.DataFrame(data=val_y, columns=['rul'])
test_y = pd.DataFrame(data=test_y, columns=['rul'])

In [102]:
train_pop = pd.concat([train_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)
val_pop = pd.concat([val_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)
test_pop = pd.concat([test_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)

In [103]:

train_df.to_csv(f'{data_location}/train_df_{data_header}.csv')
train_pop.to_csv(f'{data_location}/train_pop_{data_header}.csv')
train_y.to_csv(f'{data_location}/train_y_{data_header}.csv')

val_df.to_csv(f'{data_location}/val_df_{data_header}.csv')
val_pop.to_csv(f'{data_location}/val_pop_{data_header}.csv')
val_y.to_csv(f'{data_location}/val_y_{data_header}.csv')

test_df.to_csv(f'{data_location}/test_df_{data_header}.csv')
test_pop.to_csv(f'{data_location}/test_pop_{data_header}.csv')
test_y.to_csv(f'{data_location}/test_y_{data_header}.csv')

In [ ]:
#utils.plot_feature_distributions(df=train_df, feature_range=(0,1), figsize=(12,4))
#utils.plot_feature_distributions(df=val_df, feature_range=(0,1), figsize=(12,4))
#utils.plot_feature_distributions(df=test_df, feature_range=(0,1), figsize=(12,4))

### ~plot feature importances~

In [ ]:
# from xgboost import XGBRegressor
# importance_model = XGBRegressor()
# importance_model.fit(train_df.values, train_y)

# scoring = ['explained_variance',
#            'max_error']#,
#            #'neg_mean_absolute_error',
#            #'neg_root_mean_squared_error',
#            #'r2']

# utils.plot_feature_importances(model=importance_model,
#                                features=train_df.values,
#                                feature_labels=Xs_cols + W_cols,
#                                target=train_y,
#                                target_label='rul',
#                                scoring=scoring)

# utils.plot_feature_importance(features=train_df.values,
#                               feature_labels=Xs_cols + W_cols,
#                               target=train_y,
#                               target_label='rul',
#                               figsize=(9,4))

In [99]:
# #important_features = list(train_df.columns) #['Wf','Nc', 'T24', 'T48', 'T50', 'P2', 'Ps30', 'P40']
# important_features = ['Wf','Nf', 'T30', 'T48', 'T50', 'P15', 'P24', 'P40', 'P50','TRA', 'T2', 'time']
# important_features
len(train_df.columns)

In [117]:
X_train.shape

(279601, 100, 19)

## 2. temporalize the data

In [104]:
important_features = list(train_df.columns)

lookback = 100
horizon = 1
n_out = 1
n_features = len(important_features)

X_train, y_train = utils.temporalize_data(train_df[important_features].values, train_y.values, lookback, horizon, n_features, n_out)
#train_pop_temp, y_pop = utils.temporalize_data(train_pop.values, train_y, lookback, horizon, len(train_pop.columns), n_out)

# X_val, y_val = utils.temporalize_data(val_df.values, val_y, lookback, horizon, n_features, n_out)
# val_pop_temp, y_pop = utils.temporalize_data(val_pop.values, val_y, lookback, horizon, n_features, n_out)

# X_test, y_test = utils.temporalize_data(test_df.values, test_y, lookback, horizon, n_features, n_out)
# test_pop_temp, y_pop = utils.temporalize_data(test_pop.values, test_y, lookback, horizon, n_features, n_out)

In [105]:
np.save(f'{data_location}/{data_header}_X.npy', X_train)
np.save(f'{data_location}/{data_header}_y.npy', y_train)

In [112]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

In [118]:
f'{model_location}/{data_header}.h5'

'/home/macslab/phm2021_data_challenge/models/Fc-3_dataset-DS08.h5'

## Parameter search

In [122]:
input_shape = (lookback, n_features)
my_tuning = tuning.Tuning(input_shape, n_out)
bayesian_tuning = my_tuning.bayesian_search(objective='root_mean_squared_error',
                                            mode='min',
                                            max_trials=96,
                                            alpha=.0005,
                                            beta=5,
                                            epochs=5,
                                            executions_per_trial=1,
                                            hypermodel=my_tuning.create_bilstm_hypermodel,
                                            directory=f'{log_location}/{data_header}_v2',
                                            project_name='bilstm',
                                            logger=TensorBoardLogger(
                                                metrics=['root_mean_squared_error'],
                                                         logdir=f'{log_location}/{data_header}_v2/hparams'
                                            ),
                                            X=X_train,
                                            y=y_train)

Trial 96 Complete [00h 06m 37s]
root_mean_squared_error: 2.514153480529785

Best root_mean_squared_error So Far: 2.2991015911102295
Total elapsed time: 09h 18m 57s
INFO:tensorflow:Oracle triggered exit


In [ ]:
bayesian_tuning_params = bayesian_tuning.get_best_hyperparameters(num_trials=1)[0]
bayesian_tuning_model = bayesian_tuning.get_best_models()[0]
print(bayesian_tuning_params.values)
bayesian_tuning_model.summary()

In [ ]:
bayesian_tuning_model.save(f'{model_location}/{data_header}_best.h5')
bayesian_tuning_model.save(f'{model_location}/{data_header}_best')

# scratch / misc below

In [ ]:
# Xs_prime = []
# idx = np.random.randint(len(Xs_cols))
# polys = []

# df[W_cols] = xscaler.transform(df[W_cols])
# pred = model.predict(df[W_cols])

# for asset_id in pd.unique(df.asset_id):
#     X_full = df[df.asset_id == asset_id][W_cols]
#     pred = model.predict(X_full)
#     # pred = model.predict(xscaler.transform(X_full))
#     Xs_prime.append(pred)
    
#    y_full = df[df.asset_id == asset_id][Xs_cols]
#    trace = yscaler.transform(y_full)
#     res = trace - pred

#     x = np.arange(0, len(res[:,0]))
#     x = x[0:len(x):10]
#     for i in range(0, len(Xs_cols)):
#         y = res[:,i]
#         y = y[0:len(y):10]
#         reg = np.poly1d(np.polyfit(x, y, 6))
#         polys.append(reg)

#     fig = plt.figure(figsize=(12,12))
#     filtered = np.zeros(res.shape)
#     x = np.arange(0, len(res[:,0]))
#     for j in range(len(Xs_cols)):
#         y = res[:,j]
#         p = polys[j]

#         diff = np.zeros((len(y),))
#         for i in range(len(y)):
#             predp = p(y[i])
#             diff[i] = y[i] - predp

#         mean = np.mean(diff)
#         std = np.std(diff)

#         filt = np.zeros((len(y),))
#         for i in range(len(res)):
#             if np.abs(np.abs(y[i]) - np.abs(p(x[i]))) > np.abs(mean)+3*std:
#                 filt[i] = p(x[i])
#             else:
#                 filt[i] = y[i]

#         filtered[:,j] = filt
#         utils.plot_trace_samples(fig, y, filt, p, j)
#     plt.show()
#     Xs_prime.append(pred)

In [ ]:
# import Queue
# from threading import Thread

# units = api._get_units(db=db)
# tables = ['summary_tb', 'telemetry_tb']

# def func(units, tables, downsample):
#     return str(units) + str(tables) + str(downsample)
# def delegate():
#     return func('units', ' tables', 20)
# units = api._get_units(db=db)
# data_frames = []
# for i in range(int(len(units)/10)):
    
    
    # df = api._get_data(db=db,
    #                    units=units,
    #                    tables=tables,
    #                    downsample=20).astype(np.float32)
    
    
# train_df = pd.read_csv('train_df.csv', index_col=0)
# train_pop = pd.read_csv('train_pop.csv', index_col=0)
# train_y = pd.read_csv('train_y.csv', index_col=0)

In [ ]:
from keras.utils.vis_utils import plot_model
input_shape = (lookback, n_features)
my_tuning = tuning.Tuning(input_shape, n_out)

params = tuning.MyParameters(layers=3, units=64, dropout_rate=0, learning_rate=.001, recurrent_dropout=0,l2=-1)


model = my_tuning.build_lstm_model(params)
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)
model.summary()